In [1]:
!pip install requests beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests, csv
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [3]:
class Scraper:
    def scrape_links_by_class(self, urls: list, target_class: str) -> dict:
        result = {}
        for url in urls:
            soup = self.__get_url(url)
            if soup == None:
                print(f'[scrape_links_by_class] このURLで取得結果なし: {url}')
                continue
            
            # タイトルを取得
            title = soup.title.string
            print(f'[scrape_links_by_class] ページのタイトル: {title}')

            # ドメインを取得
            domain = self.__extract_host(url)
            
            # 目標のclass属性を持つul要素をすべて取得
            target_ul = soup.find_all('ul', class_=target_class)

            # 各ul内のリンクを取得
            for ul in target_ul:
                # ul内のリンクを取得
                content = ul.find_all('a')
            
                # リンクのテキストとURLを表示
                for link in content:
                    link_text = link.text
                    link_url = link.get('href')
                    if link_url and link_text:
                        result[link_text] = domain + link_url

        return result
    
    def scraped_text_by_id(self, url:str, id:str) -> str:
            soup = self.__get_url(url)
            if soup == None:
                print(f'[scraped_content_by_id] このURLで取得結果なし: {url}')
                return ''
            
            # タイトルを取得
            title = soup.title.string
            print(f'[scraped_content_by_id] ページのタイトル: {title}')
            
            # 目標のid属性を持つdiv要素のうち、最初のdiv要素を取得
            target_div = soup.find('div', id=id)
            clean_text = target_div.get_text()
            clean_text = ' '.join(clean_text.split())
            return title + ' ' + clean_text

    def scraped_text_by_table(self, url:str) -> str:
            soup = self.__get_url(url)
            if soup == None:
                print(f'[scraped_text_by_table] このURLで取得結果なし: {url}')
                return ''
            
            # タイトルを取得
            title = soup.title.string
            print(f'[scraped_text_by_table] ページのタイトル: {title}')
            
            # table要素のうち、最初のtable要素を取得
            target_table = soup.find('table')
            clean_text = target_table.get_text()
            clean_text = ' '.join(clean_text.split())
            return title + ' ' + clean_text
    
    def __get_url(self, url: str) -> any:
        # ウェブページのコンテンツを取得
        response = requests.get(url)

        # ステータスコードが200 (成功) の場合にのみ処理を続行
        if response.status_code == 200:
            # Beautiful Soupを使用してHTMLを解析
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup                   
        else:
            print(f'[__get_url] エラーコード: {response.status_code}')
            return None
        
    def __extract_host(self, url: str) -> str:
        url = '{uri.scheme}://{uri.netloc}'.format(uri=urlparse(url))
        return url
        
scraper = Scraper()
csv_data = []

In [ ]:
# Web Pageのリンク一覧
urls_blog = []

# 目標のclass属性
# 記事一覧のクラス
target_class = 'contentsList'  

# 目的のタイトル
target_title = ''

# 記事タイトルとリンクの保存先
target_link_map, scraped_link_map = {}, {}
scraped_link_map = scraper.scrape_links_by_class(urls_blog, target_class)

for text, url in scraped_link_map.items():
    if target_title in text:
        target_link_map[text] = url

print('対象リンク一覧 ---')
for text, url in target_link_map.items():
    print(f'リンクテキスト: {text}')
    print(f'リンクURL: {url}')

In [ ]:
# ウェブページのコンテンツを取して、csv_dataに格納
target_id = ''

for title, url in target_link_map.items():
    print(title, url)
    text = scraper.scraped_text_by_id(url, target_id)
    print(len(text), text)
    csv_data.append(text)

In [10]:
# csvに落とし込む
csv_path = '../data/data.csv'
print(len(csv_data))

csv_raw_data = []

for data in csv_data:
    csv_raw_data.append([data, ])

# CSVファイルへの書き込み
with open(csv_path, mode='w', encoding='UTF-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_raw_data)

18
